In [ ]:
import pandas as pd
import numpy as np

In [13]:
P = pd.read_csv("cap_cal_data.csv")
P.head()

,Unnamed: 0,ADEP,ADES,FiledOBT,FiledAT,ACType,ACOperator,ArrivalDelay,DepartureDelay,month,weekday,filedATminutes,filedOBTminutes
0,420,UUDD,EGLL,2019-03-01 02:40:00,2019-03-01 06:15:15,A321,BAW,23.0,12.0,3,4,375,160
1,1173,UUEE,EGLL,2019-03-01 05:00:00,2019-03-01 08:36:02,A321,AFL,25.0,10.0,3,4,516,300
2,1661,UUEE,EGLL,2019-03-01 05:25:00,2019-03-01 08:59:34,B738,AFL,27.0,11.0,3,4,539,325
3,1687,LFLL,EGLL,2019-03-01 05:29:00,2019-03-01 06:50:53,A320,BAW,17.0,7.0,3,4,410,329
4,1945,LOWW,EGLL,2019-03-01 05:40:00,2019-03-01 07:43:12,A321,AUA,8.0,6.0,3,4,463,340


In [ ]:
k = P.groupby()